### *Профиль биология*

Набор данных включает 171 молекулу, предназначенную для функциональных доменов белка CRY1, ответственного за формирование циркадного ритма. 56 молекул токсичны, а остальные нетоксичны.

Полученные данные представляют собой полный набор из 1203 молекулярных дескрипторов.

Данное домашнее задание имеет свободную форму, то есть вашей задачей будет для каждого класса моделей получить лучший результат и после выбрать наилучшую модель. Метрика для задачи - `from sklearn.metrics import f1_score`. Работа будет оцениваться по следующим ключевым пунктам:


1.   Предвартельный анализ данных
2.   Предобработка данных
  1.   Обработка пропусков
  2.   Обработка выбросов

3.   Реализация моделей
  1.  Дерево
  2.  Лес
  3.  Логистическая регрессия
  4.  KNN
  5.  MLP



## Предварительный анализ данных

In [ ]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge, Lasso, ElasticNet
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
import xgboost as xgb
from IPython.display import clear_output

from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import make_scorer, accuracy_score, classification_report, f1_score, mean_absolute_percentage_error
from sklearn.model_selection import GridSearchCV

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
plt.style.use('classic')

import warnings
warnings.filterwarnings("ignore")

Исходя из описания, наша задача будет состоять в классификации молекул на токсичные и нетоксичные на основе их молекулярных дескрипторов. Следовательно, мы будем решать задачу бинарной классификации.

В этом разделе ожидается, что вы посмотрите на распределение признаков и классов, изучите их связь, проанализируете признаки на наличие в них пропусков или выбросов.

In [ ]:
df = pd.read_csv('/content/data_bio.csv')

Посмотрим на первые строки этой таблицы:

In [ ]:
df.head()

Посмотрим на типы данных в нашей таблице:

In [ ]:
df.info()

Посмотрим на описательные стаистики:

In [ ]:
df.describe()

Проверим, имеются ли в нашем данных пропуски. Если да, то удалим их:

In [ ]:
df.isnull().sum()

Видно, что таргетная переменная `Class` является категориальной. Заменим `NonToxic` на 0, а `Toxic` на 1:

In [ ]:
df['Class_Toxic'] = df['Class'].map({'NonToxic': 0, 'Toxic': 1})
df.drop('Class', axis = 1, inplace = True)
df.head()

Так как наш датасет содержит большое число фичей, то давайте с помощью модели случайного леса отберем 10 наиболее важных и продолжим работать с ними:

In [ ]:
features = df.drop('Class_Toxic', axis = 1)
target = df['Class_Toxic']

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size = 0.2, random_state = 42)

rf_classifier = RandomForestClassifier(n_estimators = 100, random_state = 42)
rf_classifier.fit(X_train, y_train)

feature_importances = pd.DataFrame({'Feature': features.columns, 'Importance': rf_classifier.feature_importances_})
feature_importances_10 = feature_importances.sort_values(by = 'Importance', ascending = False)[:10]

print(feature_importances_10['Feature'])

Преобразуем датасет:

In [ ]:
#imp_feat = ["ATSC8c", "MATS1e", "minsCH3", "MATS4e", "MATS4s", "ATSC7i", "SpMin4_Bhp", "MLFER_S", "ATSC4p", "SpMax2_Bhm"] # признаки из статьи
imp_feat = ['ATSC7p', 'SpMax_Dt', 'MDEC-23', 'SpMax3_Bhi', 'ZMIC1', 'ATSC1v', 'MWC2', 'TIC0', 'MATS7p', 'SpMax8_Bhi']
data = df[imp_feat]
data['Class_Toxic'] = df['Class_Toxic']
data

Посмотрим на гистограммы и ядерные оценки плотности для всех признаков из датасета отдельно для каждого класса:

In [ ]:
sns.set_style("whitegrid")

for feature in data.columns[:-1]:

    fig, axs = plt.subplots(1, 2, figsize = (12, 5))
    plt.subplots_adjust(wspace = 0.5)

    sns.histplot(data = data, x = feature, hue = 'Class_Toxic', alpha = 0.5, ax = axs[0])
    axs[0].set_title(f'Гистограмма для {feature}', fontsize = 10)
    axs[0].set_facecolor('white')
    axs[0].grid(True)

    sns.kdeplot(data = data, x = feature, hue = 'Class_Toxic', common_norm = True, ax = axs[1])
    axs[1].set_title(f'KDE для {feature}', fontsize = 10)
    axs[1].set_facecolor('white')
    axs[1].grid(True)

**Выводы:**

1) Во всех графиках виден дисбаланс между классами.

2) Из KDE видно, что распределения стремятся к нормальному.

3) Также видно, что для большинства признаков, максимумы распредения плотности совпадают.

Из гистограмм и KDE видно, что выбросы в даннных присутсвуют. Убедимся в этом, используя `box-plot`:

In [ ]:
plt.figure(figsize = (20, 40))

num_rows = 10
num_cols = 5

for i, feature in enumerate(imp_feat, 1):
    plt.subplot(num_rows, num_cols, i)
    sns.boxplot(data = data[feature], color = 'skyblue', width = 0.5)

    plt.title(f'{feature}', fontsize = 15)
    plt.xlabel('Count', fontsize = 15)
    plt.ylabel(None)

    plt.xticks(fontsize = 12)
    plt.yticks(fontsize = 12)
    plt.grid(True)

plt.tight_layout()

**Выводы:** Теперь выбросы стали более наглядными.



## Предобработка данных

В этом разделе ожидается, что вы обработаете пропуски (если они есть) с помощью линейной регрессии, а выбросы (если они есть) обработайте способом на ваше усмотрение. Для оценки качества обработки пропусков используйте метрику MAPE.

Создадим массив признаков и массив таргета. Разобьем наши данные на обучающую и тестовую выборки в отношении 7:3:

In [ ]:
X = df.drop('Class_Toxic', axis = 1)
y = df['Class_Toxic']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

Для борьбы с выбросами применим стандартизацию данных:

In [ ]:
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

Давайте посмотрим на распределение наших данных по целевой переменной по всему датасету, тренировочной и тестовой выборках:

In [ ]:
original = df['Class_Toxic'].value_counts()
train = y_train.value_counts()
test = y_test.value_counts()

fig, axes = plt.subplots(1, 3, figsize = (15, 5))
sns.barplot(x = original.index, y = original.values, ax = axes[0], palette = ['blue'])
axes[0].set_title('Распределение классов в df')
axes[0].set_ylabel('Количество')

sns.barplot(x = train.index, y = train.values, ax = axes[1], palette = ['green'])
axes[1].set_title('Распределение классов в train')

sns.barplot(x = test.index, y = test.values, ax = axes[2], palette = ['orange'])
axes[2].set_title('Распределение классов в test')

Теперь видно, что между классами сильный перекос.

## Обучение моделей

Для лучшей точности перед обучением каждой модели будем делать поиск по сетке. Также не будем забывать про дисбаланс классов. Все параметры для GreadSearchCV были взяты из документации sklearn.

### DecisionTreeClassifier

In [ ]:
param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 5, 10, 15, 20],
    'min_samples_split': [2, 5, 10, 15],
    'min_samples_leaf': [1, 2, 4, 6],
    'class_weight': ['balanced']
}

tree_classifier = DecisionTreeClassifier(random_state = 42)

# pos_label = 1 - учитывает дисбаланс классов при использовании F1-score
scorer = make_scorer(f1_score, pos_label = 1)
grid_search = GridSearchCV(estimator=tree_classifier, param_grid = param_grid, cv = 5, scoring = scorer)

grid_search.fit(X_train, y_train)

print('Наилучшие параметры:', grid_search.best_params_)
print(f'F1 с учетом сбалансированных классов: {grid_search.best_score_:.2f}')

### RandomForestClassifier

In [ ]:
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 5, 10, 15],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'class_weight': ['balanced']
}

forest_classifier = RandomForestClassifier(random_state = 42)

scorer = make_scorer(f1_score, pos_label = 1)
grid_search = GridSearchCV(estimator = forest_classifier, param_grid = param_grid, cv = 5, scoring = scorer)

grid_search.fit(X_train, y_train)

print('Наилучшие параметры:', grid_search.best_params_)
print(f'F1 с учетом сбалансированных классов: {grid_search.best_score_:.2f}')

### LogisticRegression

In [ ]:
param_grid = {
    'penalty': ['l1', 'l2'],
    'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'class_weight': ['balanced']
}

logreg_classifier = LogisticRegression(random_state = 42, solver = 'liblinear')

scorer = make_scorer(f1_score, pos_label = 1)
grid_search = GridSearchCV(estimator = logreg_classifier, param_grid = param_grid, cv = 5, scoring = scorer)

grid_search.fit(X_train, y_train)

print("Наилучшие параметры:", grid_search.best_params_)
print(f'F1 с учетом сбалансированных классов: {grid_search.best_score_:.2f}')

### KNeighborsClassifier

In [ ]:
param_grid = {
    'n_neighbors': [2, 3, 4, 5, 6, 7, 8, 9],
    'weights': ['uniform', 'distance'],
    'p': [1, 2],
}

knn_classifier = KNeighborsClassifier()

scorer = make_scorer(f1_score, pos_label = 1)
grid_search = GridSearchCV(estimator = knn_classifier, param_grid = param_grid, cv = 5, scoring = scorer)

grid_search.fit(X_train, y_train)

print("Наилучшие параметры:", grid_search.best_params_)
print(f'F1 с учетом сбалансированных классов: {grid_search.best_score_:.2f}')

### Нейронная сеть

В этом разделе предлагается самостоятельно создать и обучить нейронную сеть, про интересные структуры вы можете посмотреть на последней странице статьи.

Напишем функцию для отрисовки кривых обучения:

In [ ]:
def plot_learning_curves(history):
    fig, axs = plt.subplots(1, 2, figsize = (20, 7), facecolor = 'white')

    axs[0].plot(history['loss_train'], label = 'train')
    axs[0].plot(history['loss_val'], label = 'test')
    axs[0].set_title('Loss')
    axs[0].set_xlabel('Epoch')
    axs[0].set_ylabel('Loss')
    axs[0].legend()

    axs[1].plot(history['metric_train'], label = 'train')
    axs[1].plot(history['metric_val'], label = 'test')
    axs[1].set_title('F1 Score')
    axs[1].set_xlabel('Epoch')
    axs[1].set_ylabel('F1 Score')
    axs[1].legend()

    plt.show()

Зададим модель:

In [ ]:
class Net(nn.Module):
    def __init__(self, input_size):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, 16)
        self.fc2 = nn.Linear(16, 2)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = torch.relu(self.fc2(x))
        x = self.dropout(x)
        return x

class_weights = torch.FloatTensor([1, 10])
criterion = nn.CrossEntropyLoss(weight = class_weights)
model = Net(input_size = X_train.shape[1])

optimizer = optim.Adam(model.parameters(), lr = 0.01, weight_decay = 0.001)

num_epochs = 100
batch_size = 16
history = {'loss_train': [], 'loss_val': [], 'metric_train': [], 'metric_val': []}

for epoch in range(num_epochs):
    model.train()
    for i in range(0, len(X_train), batch_size):
        X_batch = torch.FloatTensor(X_train[i:i+batch_size])
        y_batch = torch.LongTensor(y_train[i:i+batch_size].values)

        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    model.eval()
    with torch.no_grad():
        outputs_train = model(torch.FloatTensor(X_train))
        outputs_test = model(torch.FloatTensor(X_test))

        _, predicted_train = torch.max(outputs_train, 1)
        _, predicted_test = torch.max(outputs_test, 1)

        loss_train = criterion(outputs_train, torch.LongTensor(y_train.values))
        loss_test = criterion(outputs_test, torch.LongTensor(y_test.values))

        metric_train = f1_score(y_train, predicted_train, average = 'weighted')
        metric_test = f1_score(y_test, predicted_test, average = 'weighted')

        history['loss_train'].append(loss_train.item())
        history['loss_val'].append(loss_test.item())
        history['metric_train'].append(metric_train)
        history['metric_val'].append(metric_test)

        clear_output(wait = True)
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss_train: {loss_train.item():.2f}, Loss_val: {loss_test.item():.2f}, Metric_train: {metric_train:.2f}, Metric_val: {metric_test:.2f}')

plot_learning_curves(history)

Тестирование:

In [ ]:
with torch.no_grad():
    outputs = model(torch.FloatTensor(X_test))
    test_loss = criterion(outputs, torch.LongTensor(y_test.values))
    _, predicted_test = torch.max(outputs, 1)
    test_metric = f1_score(y_test, predicted_test, average = 'weighted')
    print(f"Test Loss: {test_loss.item():.2f}, F1-score: {test_metric:.2f}")

## Анализ полученных результатов и выводы по задаче

### Предобработка данных

В процессе анализа данных мы обнаружили следующее:

1. В основном признаки имеют распределение, близкое к нормальному.

2. Присутствует значительное количество выбросов, которые были обработаны с использованием StandardScaler.

3. В данных наблюдается дисбаланс между классами.

### Обучение моделей

При анализе данных решающим деревом были выделены 10 наиболее влиятельных признаков для последующего обучения. Затем был выполнен поиск по сетке параметров для каждой модели с целью достижения наилучшей точности. Однако метрики моделей оказались недостаточно высокими. Попытка воспроизвести подход, использованный в статье, не принесла улучшений в результатах.

Далее мы решили провести обучение моделей на всем наборе данных, учитывая дисбаланс классов и подбирая параметры заранее. Следует отметить, что авторы статьи использовали Accuracy в качестве целевой метрики и достигли лучших результатов с помощью модели градиентного бустинга.

Метрики полученных моделей:

1. **Decision Tree:** 0.47
2. **Random Forest:** 0.27
3. **Logistic Regression:** 0.37
4. **K-Nearest Neighbors:** 0.40
5. **Multi-Layer Perceptron (MLP):** 0.64

### Общий вывод по задаче

Из полученных результатов видно, что наилучший показатель метрики F1 Score достигнут при использовании Multi-Layer Perceptron (MLP) модели. Таким образом, для данной задачи модель MLP является наилучшим выбором, обеспечивая наивысшую точность классификации молекул на токсичные и нетоксичные.

Конечно, полученные показатели далеки от идеала. Что могло привести к этому:

1. Неравномерное распределение классов существенно повлияло на результативность модели. Вероятно, я не учел этот дисбаланс должным образом при обучении.

2. Возможно, недостаточное количество данных не позволило моделям достаточно эффективно обучиться.

3. Также, вероятно, мне следовало провести более глубокий анализ данных перед обучением.

### Комментарий

Задача дейсвтительно классная, мне понравилось :) Только вот обидно, что в учебном курсе уделили всего лишь два семинара предобработке данных при дисбалансе классов. Возможно, из-за этого я не смог решить задачу так, как хотелось бы.